In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/retinal-disease-classification")

print("Path to dataset files:", path)

In [ ]:
import numpy as np
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

In [ ]:
base_dir = "retinal-disease-classification"
train_dir = os.path.join(base_dir, "Training_Set/Training")
train_csv = os.path.join(base_dir, "Training_Set/RFMiD_Training_Labels.csv")

df_train = pd.read_csv(train_csv)
print("Training Dataset Columns:\n", df_train.columns)
print("\nFirst Few Rows of Training Labels:\n", df_train.head())

for i in range(5):
    img_name = str(df_train['ID'][i]) + ".png"
    img_path = os.path.join(train_dir, img_name)
    
    if os.path.exists(img_path):
        img = Image.open(img_path)
        plt.imshow(img)
        plt.title(f"Label: {df_train['Disease_Risk'][i]}")
        plt.axis('off')
        plt.show()
    else:
        print(f"Image {img_name} not found in {train_dir}")


In [ ]:
# Set target image size
img_size = (128, 128)

def preprocess_images(df, image_dir):
    images = []
    labels = []
    for i in range(len(df)):
        img_name = str(df['ID'][i]) + ".png"
        img_path = os.path.join(image_dir, img_name)
        
        if os.path.exists(img_path):
            img = Image.open(img_path).resize(img_size)
            img_array = img_to_array(img) / 255.0
            images.append(img_array)
            labels.append(df['Disease_Risk'][i])
    
    return np.array(images), np.array(labels)

X_train, y_train = preprocess_images(df_train, train_dir)
print("Processed Training Data Shape:", X_train.shape, y_train.shape)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("Training Set Shape:", X_train.shape, y_train.shape)
print("Validation Set Shape:", X_val.shape, y_val.shape)

In [ ]:
img_shape = (128, 128, 3)
latent_dim = 100  # Dimensionality of the random noise input for the generator

# Generator
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(8 * 8 * 256, activation="relu", input_dim=latent_dim),
        layers.Reshape((8, 8, 256)),
        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding="same", activation="relu"),
        layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding="same", activation="relu"),
        layers.Conv2DTranspose(32, kernel_size=4, strides=2, padding="same", activation="relu"),
        layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding="same", activation="tanh")
    ])
    return model

# Discriminator
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(32, kernel_size=4, strides=2, padding="same", input_shape=img_shape),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dense(1, activation="sigmoid")
    ])
    return model


In [ ]:
discriminator = build_discriminator()
discriminator.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss="binary_crossentropy", metrics=["accuracy"])

generator = build_generator()

# GAN model (generator + discriminator)
discriminator.trainable = False
gan_input = layers.Input(shape=(latent_dim,))
fake_image = generator(gan_input)
gan_output = discriminator(fake_image)
gan = tf.keras.Model(gan_input, gan_output)
gan.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss="binary_crossentropy")


In [ ]:
epochs = 1000
batch_size = 32
sample_interval = 1000

X_train_scaled = (X_train * 2) - 1

for epoch in range(epochs):
    idx = np.random.randint(0, X_train_scaled.shape[0], batch_size)
    real_imgs = X_train_scaled[idx]

    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    fake_imgs = generator.predict(noise)

    real_labels = np.ones((batch_size, 1))
    fake_labels = np.zeros((batch_size, 1))
    
    d_loss_real = discriminator.train_on_batch(real_imgs, real_labels)
    d_loss_fake = discriminator.train_on_batch(fake_imgs, fake_labels)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    g_loss = gan.train_on_batch(noise, real_labels)

    if epoch % sample_interval == 0:
        print(f"{epoch} [D loss: {d_loss[0]:.4f}, acc.: {100*d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")


In [ ]:
import matplotlib.pyplot as plt

def generate_images(generator, num_images):
    noise = np.random.normal(0, 1, (num_images, latent_dim))
    gen_images = generator.predict(noise)
    gen_images = (gen_images + 1) / 2.0

    for i in range(num_images):
        plt.imshow(gen_images[i])
        plt.axis('off')
        plt.show()

generate_images(generator, 5)